In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../")
from CanteraTools import *
from Particle import Particle
import os
import cantera as ct
from BatchPaSR import LiME
ct.suppress_thermo_warnings(True)
# from tqdm import tqdm

In [3]:
from numbers import Number
from collections import Set, Mapping, deque

try: # Python 2
    zero_depth_bases = (basestring, Number, xrange, bytearray)
    iteritems = 'iteritems'
except NameError: # Python 3
    zero_depth_bases = (str, bytes, Number, range, bytearray)
    iteritems = 'items'

def getsize(obj_0):
    """Recursively iterate to sum size of object & members."""
    _seen_ids = set()
    def inner(obj):
        obj_id = id(obj)
        if obj_id in _seen_ids:
            return 0
        _seen_ids.add(obj_id)
        size = sys.getsizeof(obj)
        if isinstance(obj, zero_depth_bases):
            pass # bypass remaining control flow and return
        elif isinstance(obj, (tuple, list, Set, deque)):
            size += sum(inner(i) for i in obj)
        elif isinstance(obj, Mapping) or hasattr(obj, iteritems):
            size += sum(inner(k) + inner(v) for k, v in getattr(obj, iteritems)())
        # Check for custom object instances - may subclass above too
        if hasattr(obj, '__dict__'):
            size += inner(vars(obj))
        if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
            size += sum(inner(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s))
        return size
    return inner(obj_0)

In [4]:
import sys
import inspect

def get_size(obj, seen=None):
    """Recursively finds size of objects in bytes"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if hasattr(obj, '__dict__'):
        for cls in obj.__class__.__mro__:
            if '__dict__' in cls.__dict__:
                d = cls.__dict__['__dict__']
                if inspect.isgetsetdescriptor(d) or inspect.ismemberdescriptor(d):
                    size += get_size(obj.__dict__, seen)
                break
    if isinstance(obj, dict):
        size += sum((get_size(v, seen) for v in obj.values()))
        size += sum((get_size(k, seen) for k in obj.keys()))
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum((get_size(i, seen) for i in obj))
        
    if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
        size += sum(get_size(getattr(obj, s), seen) for s in obj.__slots__ if hasattr(obj, s))
        
    return size

In [2]:
gas = ct.Solution("Klippenstein.cti",name="Klippenstein.cti")
gas2 = ct.Solution("gri30.cti")
gas3 = ct.Solution("Klippenstein.cti")

In [9]:
gas.species()[0].composition

SyntaxError: invalid syntax (<ipython-input-9-57abcca2d568>, line 1)

In [ ]:
gas.species()[0].composition.get

In [2]:
p = Particle("gri30.xml", P=25*ct.one_atm)

In [11]:
isinstance(p, ct.Solution)

True

In [13]:
p+p

TypeError: unsupported operand type(s) for +: 'Particle' and 'Particle'

In [ ]:
ct.suppress_thermo_warnings()
pList = [Particle("Klippenstein.cti") for i in range(0,3000)]

### Initialization times without CPR and ReactorNet

In [16]:
%timeit -n10 gas1 = ct.Solution("Klippenstein.cti")
%timeit -n10 gas2 = ct.Solution("gri30.xml")
%timeit -n10 p1 = Particle("Klippenstein.cti")
%timeit -n10 p2 = Particle("gri30.xml")

740 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.8 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
744 ms ± 5.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.9 ms ± 766 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Memory Usage without CPR and ReactorNet

In [7]:
gas1 = ct.Solution("Klippenstein.cti")
gas2 = ct.Solution("gri30.xml")
p1 = Particle("Klippenstein.cti")
p2 = Particle("gri30.xml")

print(list(map(getsize, [gas1,gas2,p1,p2])))
print(list(map(get_size, [gas1,gas2,p1,p2])))

[112, 112, 10750, 4471]
[112, 112, 10750, 4471]


## Initialization Times with CPR and ReactorNet

In [32]:
%timeit -n10 gas1 = ct.Solution("Klippenstein.cti")
%timeit -n10 gas2 = ct.Solution("gri30.xml")
%timeit -n10 p1 = Particle("Klippenstein.cti")
%timeit -n10 p2 = Particle("gri30.xml")

738 ms ± 9.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
18.9 ms ± 92.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
736 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
18.7 ms ± 428 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
gas1 = ct.Solution("Klippenstein.cti")
gas2 = ct.Solution("gri30.xml")
p1 = Particle("Klippenstein.cti")
p2 = Particle("gri30.xml")

print(list(map(getsize, [gas1,gas2,p1,p2])))
print(list(map(get_size, [gas1,gas2,p1,p2])))

This is the Particle constructor
This is the Particle constructor
[112, 112, 3812, 2237]
[112, 112, 3972, 2397]


## Memory Usage without timeHistory_list, with timeHistory_array instead

In [8]:
bp = LiME("gri30.xml", P=25*ct.one_atm, N_MAX=1000)

In [9]:
bp()


  gri30:

       temperature             300  K
          pressure     2.53313e+06  Pa
           density         2.04723  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           26470        5.336e+04     J
   internal energy     -1.2109e+06       -2.441e+06     J
           entropy           51638        1.041e+05     J/K
    Gibbs function     -1.5465e+07       -3.118e+07     J
 heat capacity c_p           14312        2.885e+04     J/K
 heat capacity c_v           10187        2.054e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -12.4984
     [  +52 minor]              0                0



In [7]:
bp2 = LiME.from_gas(p1, "Klippenstein.cti")

NameError: name 'p1' is not defined

In [20]:
gas1.chemistry_enabled

[autoreload of Particle failed: Traceback (most recent call last):
  File "C:\Users\Edwin\Anaconda3\envs\intelEnv\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Edwin\Anaconda3\envs\intelEnv\lib\site-packages\IPython\extensions\autoreload.py", line 368, in superreload
    module = reload(module)
  File "C:\Users\Edwin\Anaconda3\envs\intelEnv\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\Edwin\Anaconda3\envs\intelEnv\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "..

AttributeError: 'Solution' object has no attribute 'chemistry_enabled'

In [24]:
p_n = Particle("gri30.xml", P=25*ct.one_atm)

In [3]:
bp.TPX

(300.0,
 101324.99999999999,
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]))

In [11]:
gas1 = ct.Solution("Klippenstein.cti")
gas2 = ct.Solution("gri30.xml")
p1 = Particle("Klippenstein.cti")
p2 = Particle("gri30.xml")

print(list(map(getsize, [gas1,gas2,p1,p2])))
print(list(map(get_size, [gas1,gas2,p1,p2])))

NameError: name 'getsize' is not defined